# 본론. 데이터 준비와 분석 및 시각화를 위한 전처리

In [ ]:
# 데이터 전처리 기본 라이브러리
import pandas as pd
import numpy as np
import glob
import math

# 그래프 시각화 라이브러리
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rc, font_manager

# 맵 시각화 라이브러리
import folium
from folium import plugins
from folium.features import CustomIcon
import json
import requests

### 교통량 데이터 지점 번호별, 유입/유출 별 분류하기

In [ ]:
# 년도와 월별로 구분되어있는 파일들 불러오기
path = r'./CSV/년도별_total'
filenames = glob.glob(path + "/*.csv")
dfs = []
point = 'F-09'

# 루프로 각 지점 번호에 대한 데이터들 빼오기
for filename in filenames:
    df = pd.read_csv(filename, 
                     header = 0)
    df = df[df['지점번호'] == point]
    df = df.loc[:, ['일자', '요일', '지점번호', '구분', '방향', 
                    'time_1', 'time_2', 'time_3', 'time_4', 
                    'time_5', 'time_6', 'time_7', 'time_8', 
                    'total_day']]
    dfs.append(df)
    
# 만들어진 프레임 합치고 유입과 유출을 구분하여 새로운 데이터 프레임 생성
all_tables = pd.concat(dfs, axis = 0, ignore_index = True)
all_tables = all_tables.sort_values(by = '일자', ascending = True)
all_table_in = all_tables[all_tables['구분'] == '유입']
all_table_out = all_tables[all_tables['구분'] == '유출']
all_table_in = all_table_in.sort_values(by = '일자', ascending = True)
all_table_out = all_table_out.sort_values(by = '일자', ascending = True)

# 각 포인트별 토탈, 유입, 유출 세 버전으로 저장하기 
all_tables.to_csv('./CSV/' + point+'.csv')
all_table_in.to_csv('./CSV/'+point+'_IN.csv')
all_table_out.to_csv('./CSV/'+point+'_OUT.csv')

### 교통량 데이터를 원하는 시간대별로 묶어 년도별, 월별 저장하기

In [ ]:
# 각 년도별 만들어진 폴더에서 모든 파일 가져와 작업 
# 큰 문제가 원본 데이터에서 년도별 월별 컬럼 명칭과 갯수 등 파일 형식이 각기 달랐기에 루프로 작업하는데 어려움이 많았음.
path = r'./CSV/2021'
filenames = glob.glob(path + "/*.csv")
dfs = []
column_name = ['일자','요일','지점명','지점번호','구분','방향',
               '0시','1시','2시','3시','4시','5시','6시','7시',
               '8시','9시','10시','11시','12시','13시','14시',
               '15시','16시','17시','18시','19시','20시','21시',
               '22시','23시']

for filename in filenames:
    df = pd.read_csv(filename, 
                     header = None, thousands = ',')
    
    df.columns = column_name
    df = df.drop([0])
    df = df.dropna(axis=0)
    dfs.append(df)
    
all_tables = pd.concat(dfs, axis = 0, ignore_index = True)

# 1시간 단위로 보는것 보다 세시간 단위로 끊어 보는것이 편할 것 같아 타임 프레임을 새로 지정해줌
time1 = ['0시','1시','2시']
time2 = ['3시','4시','5시']
time3 = ['6시','7시','8시']
time4 = ['9시','10시','11시']
time5 = ['12시','13시','14시']
time6 = ['15시','16시','17시']
time7 = ['18시','19시','20시']
time8 = ['21시','22시','23시']
time_total = ['0시','1시','2시', '3시','4시','5시', 
              '6시','7시','8시', '9시','10시','11시', 
             '12시','13시','14시', '15시','16시','17시',
             '18시','19시','20시', '21시','22시','23시']

# 각 타임 별, 각 날짜에 따른 합계를 구하기
all_tables['time_1'] = all_tables[time1].sum(axis = 1)
all_tables['time_2'] = all_tables[time2].sum(axis = 1)
all_tables['time_3'] = all_tables[time3].sum(axis = 1)
all_tables['time_4'] = all_tables[time4].sum(axis = 1)
all_tables['time_5'] = all_tables[time5].sum(axis = 1)
all_tables['time_6'] = all_tables[time6].sum(axis = 1)
all_tables['time_7'] = all_tables[time7].sum(axis = 1)
all_tables['time_8'] = all_tables[time8].sum(axis = 1)

# 날짜 별 총 합 구하기
all_tables['total_day'] = all_tables[time_total].sum(axis = 1)

# 년도별 파일로 저장
all_tables.to_csv('./CSV/2021_total.csv')

### 개별로 만들어진 교통량 증감율 파일들을 하나의 파일로 합치기

In [ ]:
# 유입/유출 폴더에서 모든 파일 불러오기
path = r'./CSV/traffic/OUT'
filenames = glob.glob(path + '/*.csv')
dfs = []

# 루프를 통해 파일 하나씩 불러 읽으며 한곳으로 모으기
for filename in filenames:
    df = pd.read_csv(filename, header = 0, thousands = ',')
    dfs.append(df)
    
all_IN = pd.concat(dfs, axis = 0, ignore_index = False)
all_IN = all_IN.sort_values(by = 'Point_No', ascending = True)

column_name = ['0','Point_No','P1','P2','P3','P4','P5','P6','P7','P8','P9',
               'P10','P11','P12','P13','P14','P15','P16','P17','P18',
               'P19','P20','P21','P22']
all_IN.columns = column_name

# 소수점을 깔끔하게 하기위해 두자리수 까지만 남겨놓기
all_IN['P2'] = round(all_IN['P2'],2)
all_IN['P3'] = round(all_IN['P3'],2)
all_IN['P4'] = round(all_IN['P4'],2)
all_IN['P5'] = round(all_IN['P5'],2)
all_IN['P6'] = round(all_IN['P6'],2)
all_IN['P7'] = round(all_IN['P7'],2)
all_IN['P8'] = round(all_IN['P8'],2)
all_IN['P9'] = round(all_IN['P9'],2)
all_IN['P10'] = round(all_IN['P10'],2)
all_IN['P11'] = round(all_IN['P11'],2)
all_IN['P12'] = round(all_IN['P12'],2)
all_IN['P13'] = round(all_IN['P13'],2)
all_IN['P14'] = round(all_IN['P14'],2)
all_IN['P15'] = round(all_IN['P15'],2)
all_IN['P16'] = round(all_IN['P16'],2)
all_IN['P17'] = round(all_IN['P17'],2)
all_IN['P18'] = round(all_IN['P18'],2)
all_IN['P19'] = round(all_IN['P19'],2)
all_IN['P20'] = round(all_IN['P20'],2)
all_IN['P21'] = round(all_IN['P21'],2)
all_IN['P22'] = round(all_IN['P22'],2)

# 유입과 유출을 따로 구분하여 저장
all_IN.to_csv('./CSV/traffic/유출_증감율.csv')

### 외부 사이트에서 받은 서울시와 경기도의 시/구별 인구량 데이터를 필요한 기간과 지역에 맞춰 분류 작업하기

In [ ]:
# 서울시 인구 데이터 가져오기
df = pd.read_csv('./CSV/population/서울시_주민등록인구_구별_수정.csv', header = 0, index_col = False)

# 구 이름으로 리스트 만들기
region_list = ['강남구','강동구','강북구','강서구',
                            '관악구','광진구','구로구','금천구','노원구',
                            '도봉구','동대문구','동작구','마포구','서대문구',
                            '서초구','성동구','성북구','송파구','양천구',
                            '영등포구','용산구','은평구','종로구','중구','중랑구'
                            ]

# 루프에서 사용하기 위한 빈 리스트들
r_list = []
n_list = []

# 각 구에 따른 데이터 정리해서 옮겨 담기
for region in region_list:
    r_list = df[df.isin(df[(df.자치구 == region)])]['인구']
    n_list.append(r_list)
    
# 새로운 데이터 프레임을 만들어 리스트 합쳐주기
df_new = pd.concat(n_list, axis = 1, ignore_index = True)

df_new.to_csv('./CSV/서울시_주민등록인구_구별.csv')

#-------------------------------------------------------------------

# 같은 방식으로 경기도 인구 데이터 작업
df = pd.read_csv('./CSV/population/경기도_시별_인구.csv', header = 0)

# 숫자 값들이 str으로 지정되어있기 때문에 int로 바꿔주기
for i in 0 ... 32 :
    df[i] = df[i].astype(int)

df_sub = pd.DataFrame(index = range(0,87),
                     columns = ['합계','수원시','용인시','성남시','부천시',
                                '화성시','안산시','안양시','평택시','시흥시',
                                '김포시','광주시','광명시','군포시','하남시',
                                '오산시','이천시','안성시','의왕시','양평군',
                                '여주시','과천시','고양시','남양주시','파주시',
                                '의정부시','양주시','구리시','포천시','동두천시',
                                '가평군','연천군'])

# 서울시 인구 데이터와 구성이 다르기 때문에 루프문에 조금의 차이가 있음
idx = 1
df.iloc[88,1]
period_list = []
frames = []

while idx < 33:
    for i in range(0,87,3):
        period = df.iloc[i,idx]+df.iloc[i+1,idx]+df.iloc[i+2,idx]
        period_list.append(period)
        #print(period_list)
    narray = np.array(period_list)
    #period_list2 = pd.DataFrame.from_records(narray)
    period_list2 = pd.Series(period_list)
    frames += [period_list2]
    period_list = []
    idx += 1

# 리스트에 담긴 데이터들 새로운 데이터 프레임에 합쳐주기
result = pd.concat(frames, axis = 1, ignore_index = True)

# 최종적으로 컬럼과 인덱스명 지정해주기
result.columns = columns_cities
result['분기'] = index_list

result.to_csv('./CSV/population/경기도_시별_인구_분기별.csv')

### 서울시와 경기도의 시/구별 인구량 데이터를 받아 증감율로 변환하기

In [ ]:
# 서울시 구별 그리고 2015~2021에 해당하는 분기별 인구량 데이터 받아오기
df = pd.read_csv('./CSV/population/서울시_주민등록인구_구별_수정.csv')

# 숫자에서 천의 자리 , 부호 없애주기
for i in 0 ... 28 :
    df[i] = df[i].str.replace(',','')
    
# 숫자들이 str값으로 읽어지므로 이를 float으로 변환 후 int로 변환하기
for j in 0 ... 28 :
    df[j] = df[j].astype(float).astype(int)
    
#새로운 데이터 프레임을 만들어 인덱스와 컬럼명 재설정해준 후 데이터 옮겨주기
df_sub = pd.DataFrame(index = range(1, 29), columns = ['강남구', '강동구', '강북구','강서구',
                                 '관악구','광진구','구로구','금천구','노원구',
                                 '도봉구','동대문구','동작구','마포구','서대문구',
                                 '서초구','성동구','성북구','송파구','양천구',
                                 '영등포구','용산구','은평구','종로구','중구','중랑구'])

# 인구량 데이터를 인구 증감율 데이터로 계산해주기
i = 0
while i < 25:
    for idx in range(1, 29):
        sub = df.iloc[idx,i+1] - df.iloc[idx-1,i+1]
        ratio = (sub/df.iloc[idx-1, i+1])*100
        df_sub.loc[idx, df_sub.columns[i]] = round(ratio, 2)
    i+=1

# 인구 증감율 파일 csv로 저장하기
df_sub.to_csv('./CSV/population/서울시_인구_증감율.csv')

# 같은 방식으로 경기도 인구 증감율 파일 만들기
df = pd.read_csv('./CSV/population/경기도_시별_인구_분기별.csv')

df_sub = pd.DataFrame(index = range(1,29), columns = ['합계','수원시','용인시','성남시','부천시',
                                '화성시','안산시','안양시','평택시','시흥시',
                                '김포시','광주시','광명시','군포시','하남시',
                                '오산시','이천시','안성시','의왕시','양평군',
                                '여주시','과천시','고양시','남양주시','파주시',
                                '의정부시','양주시','구리시','포천시','동두천시',
                                '가평군','연천군'])

i = 0
while i < 32:
    for idx in range(1,29):
        sub = df.iloc[idx,i+1] - df.iloc[idx-1, i+1]
        ratio = (sub/df.iloc[idx-1, i+1])*100
        df_sub.loc[idx, df_sub.columns[i]] = round(ratio, 2)
    i += 1
    
df_sub.to_csv('./CSV/population/경기도_인구_증감율.csv')

### Matplotlib를 이용해 서울시와 경기도 인구 증감율에 대한 각 년도별 그래프 그리기


* 이전에 정리된 서울시와 경기도의 년간 인구 증감율 csv 파일을 이용해 matplotlib로 그래프를 그린다. <br>
* 그래프에서 y축이 될 구 혹은 시 이름들로 새로운 그룹을 만들어주고 파일 내에서 각 년도별로 순차적으로 데이터들을 가져온다. <br>
* 한글 깨짐과 - 부호 깨짐 현상이 일어나므로 이를 방지하기 위해 맑은고딕 폰트로 미리 폰트 설정을 해주고 부호 또한 처리를 해준다. <br>
* 기간별 증감율의 범위가 매우 다른편이기에 모든 기간에 공통 x축 범위를 지정하지 못하고, 각 그래프별 데이터에 맞춰 x축의 범위를 지정하였다. <br>
* 경기도의 경우 애초에 경기통계를 통해 받은 원본 파일에 경기도 전체 합계 데이터가 들어가 있었으므로 굳이 빼지 않고 그래프에 추가하였다. <br> <br>
    

In [ ]:
# 서울시 인구 데이터 파일 불러오기
df = pd.read_csv('./CSV/population/서울시_YbyY증감율.csv')

# 구 이름으로 그래프에 들어갈 그룹 정해주기
group_names = ['강남구', '강동구', '강북구','강서구',
                                 '관악구','광진구','구로구','금천구','노원구',
                                 '도봉구','동대문구','동작구','마포구','서대문구',
                                 '서초구','성동구','성북구','송파구','양천구',
                                 '영등포구','용산구','은평구','종로구','중구','중랑구'
                                ]

# 파일 내에서 순차적으로 각 년도에 해당하는 인덱스별로 가져오기
group_data = df.iloc[1, 1:]

# 한글 깨짐 현상을 방지하기 위해 맑은고딕 폰트와 - 부호 설정
rc('font', family = "Malgun Gothic")
matplotlib.rcParams['axes.unicode_minus'] = False

# 그래프 스타일 설정
plt.style.use('fivethirtyeight')

# 그래프 그리기 x축은 증감율로, y축은 구 이름으로 설정, 년도별 증감율 폭이 다르므로 각 년도별 
# 사이즈에 맞는 x축 limit 정해주기
plt.rcParams.update({'figure.autolayout': True})
fig, ax = plt.subplots(figsize = (10, 10))
ax.barh(group_names, group_data)
labels = ax.get_xticklabels()
plt.setp(labels, rotation = 0, horizontalalignment = 'right')
ax.set(xlim = [-4, 3], xlabel = '증감율', ylabel = '구 이름', title = '서울시 인구 증감율 (2015~2016)')

# dpi 200으로 그래프 png파일로 저장
fig.savefig('./CSV/population/서울시인구증감율그래프(2015~2016).png', dpi = 200)

# -------------------------------------------------------------------------------------------

# 같은 방식으로 경기도 그래프 작업
df = pd.read_csv('./CSV/population/경기도_YbyY증감율.csv')

group_names = ['합계','수원시','용인시','성남시','부천시',
                                '화성시','안산시','안양시','평택시','시흥시',
                                '김포시','광주시','광명시','군포시','하남시',
                                '오산시','이천시','안성시','의왕시','양평군',
                                '여주시','과천시','고양시','남양주시','파주시',
                                '의정부시','양주시','구리시','포천시','동두천시',
                                '가평군','연천군']
group_data = df.iloc[7, 1:]

plt.rcParams.update({'figure.autolayout': True})
fig, ax = plt.subplots(figsize = (10, 10))
ax.barh(group_names, group_data)
labels = ax.get_xticklabels()
plt.setp(labels, rotation = 0, horizontalalignment = 'right')
ax.set(xlim = [-15, 40], xlabel = '증감율(%)', ylabel = '시 이름', title = '경기도 인구 증감율 (2016~2021)')

# dpi 200으로 경기도 그래프 png파일로 저장
fig.savefig('./CSV/population/경기도인구증감율그래프(2016~2021).png', dpi = 200)

<img src="./CSV/population/graph/서울시인구증감율그래프2016~2021.png" width="500" height="500" align="center">

<위 이미지> 서울시 2016~2021 인구 증감율의 구별 그래프
서울의 경우 지난 5년을 기준으로 봤을 때, 거의 모든 구에서 인구가 감소했고 강동구만 눈에 띄는 증가를 했음을 볼 수 있다.

<img src="./CSV/population/graph/경기도인구증감율그래프2016~2021.png" width="500" height="500" align="center">

<위 이미지> 경기도 2016~2021 인구 증감율의 시별 그래프
위 경우엔 하남, 김포, 화성시의 인구가 지난 오년 사이에 비약적으로 증가했음을 알 수있고, 
해당하는 세 지역 모두 2기와 3기 신도시 개발이 크게 일어났던 지역들이다.
#### * 모든 시기에 해당하는 그래프들은 아래에있다.

### 지도 시각화에 앞서 이전에 만들어진 지역별 인구 증감율 파일을 시각화하기 편하게끔 변환해 주기

In [ ]:
# 서울시와 경기도 인구 데이터를 불러와 열과 행의 위치를 바꿔주고 필요한 컬럼들만 남기는 작업
# Folium에서 csv파일을 받아 사용하기 위해서는 기간과 시/구가 컬럼이 되어야함
seoul_data = pd.read_csv('./CSV/population/서울시_YbyY증감율.csv', header = 0)
seoul_data = seoul_data.T
column = ['Y2015-Y2014','Y2016-Y2015','Y2017-Y2016','Y2018-Y2017',
          'Y2019-Y2018','Y2020-Y2019','Y2021-Y2020','Y2021-Y2016', 'district_name']
seoul_data = seoul_data.drop(['기간'])
seoul_data.columns = column

seoul_data.to_csv('./CSV/population/seoul_map_total.csv')

### Folium을 이용해 2016부터 2021년까지의 서울시와 경기도 인구 변화와 교통량 변화량을 보여주는 지도 만들기

* Folium 라이브러리를 이용해 서울과 경기도의 구별, 시별 인구 증감율을 지도로 나타내준다. <br>
* 서울과 경기 모두 2016년부터 2021년까지의 1년단위 layer과 16~21년의 5년단위 layer를 만들어준다. <br>
* 한국의 각 시, 구 별 경계 좌표가 들어있는 json파일을 받아와 연결해준다. <br>
* 구분을 위하여 서울시는 파란색으로 경기도는 초록색으로 명암단위 표시를 해주었다. <br>
* 교통량 데이터에서의 측정 위치를 나타내주는 마커들을 생성하고 편의를 위해 B, C, D, F points의 각각의 sub group을 만들어 담아준다. 그리고 모든 마커들을 통틀어 컨트롤 할 수 있는 큰 group안에 sub group들을 넣어준다. <br>
* 각각의 마커들은 클릭 시에 마커가 담고 있는 지점번호, 지점명 그리고 2016-2021간의 교통량 유입/유출 증감율을 보여준다. <br>
<br>

In [ ]:
# 서울시 인구 데이터 구별, 년도별로 구분하기
seoul_data = pd.read_csv('./CSV/population/seoul_map_total.csv', header = 0)

seoul_data_1 = seoul_data[['district_name', 'Y2021-Y2016']]
seoul_data_2 = seoul_data[['district_name', 'Y2021-Y2020']]
seoul_data_3 = seoul_data[['district_name', 'Y2020-Y2019']]
seoul_data_4 = seoul_data[['district_name', 'Y2019-Y2018']]
seoul_data_5 = seoul_data[['district_name', 'Y2018-Y2017']]
seoul_data_6 = seoul_data[['district_name', 'Y2017-Y2016']]

# 경기도 인구 데이터 시별, 년도별로 구분하기
gyeonggi_data = pd.read_csv('./CSV/population/gyeonggi_data_total.csv', header = 0)

gyeonggi_data_1 = gyeonggi_data[['District_Name', 'Y2021-Y2016']]
gyeonggi_data_2 = gyeonggi_data[['District_Name', 'Y2021-Y2020']]
gyeonggi_data_3 = gyeonggi_data[['District_Name', 'Y2020-Y2019']]
gyeonggi_data_4 = gyeonggi_data[['District_Name', 'Y2019-Y2018']]
gyeonggi_data_5 = gyeonggi_data[['District_Name', 'Y2018-Y2017']]
gyeonggi_data_6 = gyeonggi_data[['District_Name', 'Y2017-Y2016']]

# 한국의 각 시, 구 별 경계 좌표가 들어있는 json 파일
req = requests.get('https://raw.githubusercontent.com/southkorea/southkorea-maps/master/kostat/2013/json/skorea_municipalities_geo_simple.json')
state_geo = req.json()

# 지도의 중심을 서울 중심 좌표로 설정
seoul_map = folium.Map(location = [37.5502, 126.982],
                      zoom_start = 10.5)
                      #tiles = 'cartodbpositron')
                      #tiles = 'openstreetmap')

# Folium의 Choropleth를 이용해 위에서 나눈 각 데이터프레임들을 지도에 적용, 레이어 만들기
folium.Choropleth(
    geo_data = state_geo,
    name = '서울시 인구변화 16~21',
    data = seoul_data_1,
    columns = ['district_name','Y2021-Y2016'],
    key_on = 'feature.properties.name',
    fill_color = 'Blues',
    fill_opacity = 0.7,
    line_opacity = 0.3,
    Highlight = True,
    show = False,
    overlay = True,
    legend_name = '서울 16~21'
    ).add_to(seoul_map)

folium.Choropleth(
    geo_data = state_geo,
    name = '서울시 인구변화 20~21',
    data = seoul_data_2,
    columns = ['district_name','Y2021-Y2020'],
    key_on = 'feature.properties.name',
    fill_color = 'Blues',
    fill_opacity = 0.7,
    line_opacity = 0.3,
    Highlight = True,
    show = False,
    overlay = True,
    legend_name = '서울 20~21'
    ).add_to(seoul_map)

folium.Choropleth(
    geo_data = state_geo,
    name = '서울시 인구변화 19~20',
    data = seoul_data_3,
    columns = ['district_name','Y2020-Y2019'],
    key_on = 'feature.properties.name',
    fill_color = 'Blues',
    fill_opacity = 0.7,
    line_opacity = 0.3,
    Highlight = True,
    show = False,
    overlay = True,
    legend_name = '서울 19~20'
    ).add_to(seoul_map)

folium.Choropleth(
    geo_data = state_geo,
    name = '서울시 인구변화 18~19',
    data = seoul_data_4,
    columns = ['district_name','Y2019-Y2018'],
    key_on = 'feature.properties.name',
    fill_color = 'Blues',
    fill_opacity = 0.7,
    line_opacity = 0.3,
    Highlight = True,
    show = False,
    overlay = True,
    legend_name = '서울 18~19'
    ).add_to(seoul_map)

folium.Choropleth(
    geo_data = state_geo,
    name = '서울시 인구변화 17~18',
    data = seoul_data_5,
    columns = ['district_name','Y2018-Y2017'],
    key_on = 'feature.properties.name',
    fill_color = 'Blues',
    fill_opacity = 0.7,
    line_opacity = 0.3,
    Highlight = True,
    show = False,
    overlay = True,
    legend_name = '서울 17~18'
    ).add_to(seoul_map)

folium.Choropleth(
    geo_data = state_geo,
    name = '서울시 인구변화 16~17',
    data = seoul_data_6,
    columns = ['district_name','Y2017-Y2016'],
    key_on = 'feature.properties.name',
    fill_color = 'Blues',
    fill_opacity = 0.7,
    line_opacity = 0.3,
    Highlight = True,
    show = False,
    overlay = True,
    legend_name = '서울 16~17'
    ).add_to(seoul_map)

#-----------------------------------------------------------------------

folium.Choropleth(
    geo_data = state_geo,
    name = '경기도 인구변화 16~21',
    data = gyeonggi_data_1,
    columns = ['District_Name', 'Y2021-Y2016'],
    key_on = 'feature.properties.name',
    fill_color = 'Greens',
    fill_opacity = 0.7,
    line_opacity = 0.3,
    Highlight = True,
    show = False,
    overlay = True,
    legend_name = '경기 16~21'
    ).add_to(seoul_map)

folium.Choropleth(
    geo_data = state_geo,
    name = '경기도 인구변화 20~21',
    data = gyeonggi_data_2,
    columns = ['District_Name', 'Y2021-Y2020'],
    key_on = 'feature.properties.name',
    fill_color = 'Greens',
    fill_opacity = 0.7,
    line_opacity = 0.3,
    Highlight = True,
    show = False,
    overlay = True,
    legend_name = '경기 20~21'
    ).add_to(seoul_map)

folium.Choropleth(
    geo_data = state_geo,
    name = '경기도 인구변화 19~20',
    data = gyeonggi_data_3,
    columns = ['District_Name', 'Y2020-Y2019'],
    key_on = 'feature.properties.name',
    fill_color = 'Greens',
    fill_opacity = 0.7,
    line_opacity = 0.3,
    Highlight = True,
    show = False,
    overlay = True,
    legend_name = '경기 19~20'
    ).add_to(seoul_map)

folium.Choropleth(
    geo_data = state_geo,
    name = '경기도 인구변화 18~19',
    data = gyeonggi_data_4,
    columns = ['District_Name', 'Y2019-Y2018'],
    key_on = 'feature.properties.name',
    fill_color = 'Greens',
    fill_opacity = 0.7,
    line_opacity = 0.3,
    Highlight = True,
    show = False,
    overlay = True,
    legend_name = '경기 18~19'
    ).add_to(seoul_map)

folium.Choropleth(
    geo_data = state_geo,
    name = '경기도 인구변화 17~18',
    data = gyeonggi_data_5,
    columns = ['District_Name', 'Y2018-Y2017'],
    key_on = 'feature.properties.name',
    fill_color = 'Greens',
    fill_opacity = 0.7,
    line_opacity = 0.3,
    Highlight = True,
    show = False,
    overlay = True,    
    legend_name = '경기 17~18'
    ).add_to(seoul_map)

folium.Choropleth(
    geo_data = state_geo,
    name = '경기도 인구변화 16~17',
    data = gyeonggi_data_6,
    columns = ['District_Name', 'Y2017-Y2016'],
    key_on = 'feature.properties.name',
    fill_color = 'Greens',
    fill_opacity = 0.7,
    line_opacity = 0.3,
    Highlight = True,
    show = False,
    overlay = True,
    legend_name = '경기 16~17'
    ).add_to(seoul_map)

# 교통량 측정 지점을 담고 있는 마커들을 효율적으로 관리하기 위해 각 지점번호별 서브 그룹 만들고 
# Markers 그룹의 서브 그룹으로 부여
fg = folium.FeatureGroup(name = 'Markers 2016~2021', show = False)
seoul_map.add_child(fg)
g1 = folium.plugins.FeatureGroupSubGroup(fg, 'B_Points')
seoul_map.add_child(g1)
g2 = folium.plugins.FeatureGroupSubGroup(fg, 'C_Points')
seoul_map.add_child(g2)
g3 = folium.plugins.FeatureGroupSubGroup(fg, 'D_Points')
seoul_map.add_child(g3)
g4 = folium.plugins.FeatureGroupSubGroup(fg, 'F_Points')
seoul_map.add_child(g4)

folium.LayerControl(collapsed = True).add_to(seoul_map)

# 반복되는 코드들을 모두 표시하기엔 가독성이 떨어져 <간략>으로 표시하고 대표하는 코드들만 나타내었다.
# <간략> BeautifyIcon plugin을 이용해 각 마커의 색상과 디자인 부여하는 예시
icon_number_b1 = plugins.BeautifyIcon(
    border_color = '#FF0000', 
    text_color = '#FF0000',
    number = 1,
    inner_icon_style = 'margin-top:0'
)

# <간략> 각 지점별 마커를 만들고, 팝업으로 지점 번호, 지점명, 16~21년에 해당하는 유입, 유출량 표시하는 예시
folium.Marker(
    location = [37.6899062,127.0428593], 
    popup = folium.Popup("<b>B-1 도봉로 <br>유입:1.15% <br>유출:-15.19% </br></br></b>",max_width = 200),
    tooltip = tooltip,
    icon = icon_number_b1
    ).add_to(g1)

# 완성된 지도를 html파일로 저장
seoul_map.save('./CSV/population/sudogwon_map.html')

### 서울과 경기도의 맵 예시

<img src="./CSV/screenshots/sudogwanscreen1.png" width="500" height="500" align="left">


<위 이미지> 수도권 지도내 경기지역 16~21 인구 증감율 지도와 서울 시내 교통량 측정 지점 마커들이 켜져 있는 상태이다. <br>

<img src="./CSV/screenshots/sudogwanscreen2.png" width="500" height="500" align="left">


<위 이미지> 수도권 지도내 서울지역 16~21 인구 증감율 지도와 서울 시내 교통량 측정 지점 마커들이 켜져 있는 상태이다.
<br>

<img src="./CSV/screenshots/markerscreen1.png" width="500" height="500" align="left">


<위 이미지> 마커는 위와 같이 툴팁과 팝업 기능이 있고 툴팁을 클릭해 팝업을 띄울 시에는 지점번호, 지점명, 5년간의 유입, 유출 증감율이 나타난다. <br>
#### * 더 다양한 지도 이미지는 아래에있다.

### Folium Dual Map을 이용해 2016 ~ 2018과 2018 ~ 2021 두 기간별 인구 증감율과 교통량 증감율 비교

* 위에서 사용한 서울시와 경기도의 인구량 데이터를 사용하지만 듀얼맵의 경우에는 조금 더 명확한 차이를 보기 위해 모든 년도의 인구 증감율이 아닌 2016-2018과 2018-2021 두 구간으로 나누어 왼쪽 오른쪽으로 배치한다.
* 왼쪽은 서울시와 경기도의 2016-2018년 간의 인구 증감율과, 같은 기간동안의 교통량 지점별 증감율을 따졌을 때 가장 증가세가 높은 지점들을 골라 유입, 유출을 나누어 서클 마커로 표시하였다. 
* 오른쪽은 같은 지도이지만 기간이 2018-2021년에 해당한다.
* 서클 마커의 경우 빨간색은 유입이고 파란색은 유출이다. 각 서클 마커를 클릭하였을 때는 지점번호, 지점명, 그리고 해당 기간 동안의 증가율을 보여준다. 
* 마찬가지로 마커 관리의 편의를 위하여, 각 지도별, 유입/유출별, 지점번호별 서브 그룹을 만들어 넣어주었고, 큰 그룹으로는 유입과 유출 마커들을 관리 할 수 있도록 하였다.

In [ ]:
# 서울과 경기지역의 인구 증감율 데이터 불러오기
seoul_data = pd.read_csv('./CSV/population/seoul_map_total.csv', header = 0)
seoul_data_1 = seoul_data[['district_name', 'Y2018-Y2016']]
seoul_data_2 = seoul_data[['district_name', 'Y2021-Y2018']]

gyeonggi_data = pd.read_csv('./CSV/population/gyeonggi_data_total.csv', header = 0)
gyeonggi_data_1 = gyeonggi_data[['District_Name', 'Y2018-Y2016']]
gyeonggi_data_2 = gyeonggi_data[['District_Name', 'Y2021-Y2018']]

# 한국의 각 시, 구 별 경계 좌표가 들어있는 json 파일
req = requests.get('https://raw.githubusercontent.com/southkorea/southkorea-maps/master/kostat/2013/json/skorea_municipalities_geo_simple.json')
state_geo = req.json()

# 맵의 중심은 서울 중앙으로 설정
m = folium.plugins.DualMap(location = [37.5502, 126.982], 
                           zoom_start = 10.5,
                          tiles = 'openstreetmap')

# 듀얼맵의 각 페이지에 들어갈 서울, 경기 지역 레이어 설정
layer_1 = folium.features.Choropleth( 
    geo_data = state_geo,
    name = '서울시 인구 증감율 16~18',
    data = seoul_data_1,
    columns = ['district_name', 'Y2018-Y2016'],
    key_on = 'feature.properties.name',
    fill_color = 'Blues',
    fill_opacity = 0.8,
    line_opacity = 0.5,
    show = False,
    legend_name = '서울 16-18'
).add_to(m.m1)

layer_2 = folium.features.Choropleth( 
    geo_data = state_geo,
    name = '서울시 인구 증감율 18~21',
    data = seoul_data_2,
    columns = ['district_name', 'Y2021-Y2018'],
    key_on = 'feature.properties.name',
    fill_color = 'Blues',
    fill_opacity = 0.8,
    line_opacity = 0.5,
    show = False,
    legend_name = '서울 18-21'
).add_to(m.m2)

layer_3 = folium.features.Choropleth(
    geo_data = state_geo,
    name = '경기도 인구 증감율 16~18',
    data = gyeonggi_data_1,
    columns = ['District_Name', 'Y2018-Y2016'],
    key_on = 'feature.properties.name',
    fill_color = 'Greens',
    fill_opacity = 0.8,
    line_opacity = 0.5,
    show = False,
    legend_name = '경기 16-18'
).add_to(m.m1)

layer_4 = folium.features.Choropleth(
    geo_data = state_geo,
    name = '경기도 인구 증감율 18~21',
    data = gyeonggi_data_2,
    columns = ['District_Name', 'Y2021-Y2018'],
    key_on = 'feature.properties.name',
    fill_color = 'Greens',
    fill_opacity = 0.8,
    line_opacity = 0.5,
    show = False,
    legend_name = '경기 18-21'
).add_to(m.m2)

# folium FeatureGroup과 plugins를 이용해
# 각 기간별 교통량 유입과 유출의 가장 큰 변화가 있는 지점들만 골라내 원형 마커로 표시하기 위한 그룹핑 작업
fg1 = folium.FeatureGroup(name = '<유입> 2016~2018_Circle_Markers', show = False).add_to(m.m1)
fg2 = folium.FeatureGroup(name = '<유출> 2016~2018_Circle_Markers', show = False).add_to(m.m1)
g5 = folium.plugins.FeatureGroupSubGroup(fg1, name = '<유입> B markers').add_to(m.m1)
g6 = folium.plugins.FeatureGroupSubGroup(fg1, name = '<유입> C markers').add_to(m.m1)
g7 = folium.plugins.FeatureGroupSubGroup(fg1, name = '<유입> D markers').add_to(m.m1)
g8 = folium.plugins.FeatureGroupSubGroup(fg1, name = '<유입> F markers').add_to(m.m1)
g9 = folium.plugins.FeatureGroupSubGroup(fg2, name = '<유출> B markers').add_to(m.m1)
g10 = folium.plugins.FeatureGroupSubGroup(fg2, name = '<유출> C markers').add_to(m.m1)
g11 = folium.plugins.FeatureGroupSubGroup(fg2, name = '<유출> D markers').add_to(m.m1)
g12 = folium.plugins.FeatureGroupSubGroup(fg2, name = '<유출> F markers').add_to(m.m1)

fg3 = folium.FeatureGroup(name = '<유입> 2018~2021_Circle_Markers', show = False).add_to(m.m2)
fg4 = folium.FeatureGroup(name = '<유출> 2018~2021_Circle_Markers', show = False).add_to(m.m2)
h5 = folium.plugins.FeatureGroupSubGroup(fg3, name = '<유입> B markers').add_to(m.m2)
h6 = folium.plugins.FeatureGroupSubGroup(fg3, name = '<유입> C markers').add_to(m.m2)
h7 = folium.plugins.FeatureGroupSubGroup(fg3, name = '<유입> D markers').add_to(m.m2)
h8 = folium.plugins.FeatureGroupSubGroup(fg3, name = '<유입> F markers').add_to(m.m2)
h9 = folium.plugins.FeatureGroupSubGroup(fg4, name = '<유출> B markers').add_to(m.m2)
h10 = folium.plugins.FeatureGroupSubGroup(fg4, name = '<유출> C markers').add_to(m.m2)
h11 = folium.plugins.FeatureGroupSubGroup(fg4, name = '<유출> D markers').add_to(m.m2)
h12 = folium.plugins.FeatureGroupSubGroup(fg4, name = '<유출> F markers').add_to(m.m2)

folium.LayerControl(collapsed = True).add_to(m)

# 반복되는 코드들을 모두 표시하기엔 가독성이 떨어져 <간략>으로 표시하고 대표하는 코드들만 나타내었다.
# <간략> 원형 마커를 위한 예시 (년도와 유입/유출, 지점번호에 맞게 각 그룹에 추가)
folium.CircleMarker(
    [37.5544386,127.1083746],
    radius = 25,
    color = 'red',
    fill_color = '#ffffggg',
    popup = '<b>B-3 <br>아차산로 <br>+38.11%</br></br></b>'
).add_to(g5)

folium.CircleMarker(
    [37.6186095,127.1023901],
    radius = 20,
    color = 'blue',
    fill_color = '#ffffggg',
    popup = '<b>B-5 <br>경춘북로 <br>+28.67</br></br></b>'
).add_to(g9)

# 완성된 듀얼맵을 html로 저장
m.save('./CSV/population/dualmap.html')

### 듀얼맵 예시

<img src="./CSV/screenshots/dualmapscreen1.png" width="800" height="600" align="left
                                                                                ">


<위 이미지> 듀얼 맵에서 서울 인구 증감율 지도 2016-2018 <왼쪽>과 서울 인구 증감율 지도 2018-2021 <오른쪽> 그리고 같은 기간내 교통량 증가세가 가장 컸던 지점들의 유출 유입 마커가 켜져있는 상태이다.

#### * 더 다양한 듀얼맵 이미지는 아래에 있다.

## 모든 년도별 / 서울시와 경기도 그래프 모음

<img src="./CSV/population/graph/서울시인구증감율그래프2016~2017.png" width="380" height="350" align="left">
<img src="./CSV/population/graph/서울시인구증감율그래프2017~2018.png" width="380" height="350" align="right">

<img src="./CSV/population/graph/서울시인구증감율그래프2018~2019.png" width="380" height="350" align="left">
<img src="./CSV/population/graph/서울시인구증감율그래프2019~2020.png" width="380" height="350" align="right">

<img src="./CSV/population/graph/서울시인구증감율그래프2020~2021.png" width="380" height="350" align="left">
<img src="./CSV/population/graph/서울시인구증감율그래프2016~2021.png" width="380" height="350" align="right">


<img src="./CSV/population/graph/경기도인구증감율그래프2016~2017.png" width="380" height="350" align="left">
<img src="./CSV/population/graph/경기도인구증감율그래프2017~2018.png" width="380" height="350" align="right">

<img src="./CSV/population/graph/경기도인구증감율그래프2018~2019.png" width="380" height="350" align="left">
<img src="./CSV/population/graph/경기도인구증감율그래프2019~2020.png" width="380" height="350" align="right">

<img src="./CSV/population/graph/경기도인구증감율그래프2020~2021.png" width="380" height="350" align="left">
<img src="./CSV/population/graph/경기도인구증감율그래프2016~2021.png" width="380" height="350" align="right">